# Notebook 1

You can get the notebook file [here](https://github.com/mila-iqia/SARC/blob/master/docs/notebooks/notebook_1_sarc_connection_and_users_list.ipynb).

## Connect to SARC virtual machine

For this example, and more generally for any use of SARC, you need to access SARC virtual machine in normal (read-only) mode, with config file `config/sarc-client.json`.

In this config file, it is assumed you have access to mongoDB via local port 8123. 

First, make sure you have set up the Mila IDT VPN.
If you never accessed the VM, [see documention here to start](https://mila-iqia.atlassian.net/wiki/spaces/IDT/pages/2325840018/VM+sarc01-dev).

Then, create the SSH tunnel in a separate terminal, by forwarding the local port indicated in config file `config/sarc-client.json` (here `8123`) to the distant VM port (`21017`);

The VM is accessible through a login node. 
The easiest way is to create a specific section in your `~/.ssh/config` file:
```
Host sarc
    Hostname sarc01-dev
    User <your_mila_id>
    Proxyjump mila
    LocalForward 8123 127.0.0.1:27017
```
And then type this command in a separate terminal:
```
ssh sarc
```

## Configure SARC

To run this example, we must set environment variable `SARC_CONFIG` to configuration file `sarc-client.json`.

In a terminal opened in SARC repository folder, you can set the variable with `export SARC_CONFIG=config/sarc-client.json`.

From this notebook, you can set the variable using Python's `os.environ` object:

In [ ]:
import os;
# As we're in folder `docs/notebooks`, we must go back twice in tree (`../../`) to find config folder.
os.environ["SARC_CONFIG"] = "../../config/sarc-client.json";
print(os.environ["SARC_CONFIG"]);

## Get users list

You can get users list with SARC function `get_users`:

In [ ]:
from sarc.client import get_users
users = get_users()
print(f"Number users: {len(users)}")

You can filter uses list to get only active users:

In [ ]:
users = [u for u in users if u.mila.active]
print(f"Number of active users: {len(users)}")

## Create supervisors list from users list

You can get user's supervisor and co-supervisor in `user.mila_ldap["supervisor"]` and `user.mila_ldap["co_supervisor"]`. Note that both fields may be `None` if unknown:

In [ ]:
supervisors = set()
co_supervisors = set()
for user in users:
    supervisor = user.mila_ldap.get("supervisor")
    co_supervisor = user.mila_ldap.get("co_supervisor")
    if supervisor:
        supervisors.add(supervisor)
    if co_supervisor:
        co_supervisors.add(co_supervisor)
print(f"Number of supervisors: {len(supervisors)}")
print(f"Number of co-supervisors: {len(co_supervisors)}")

## Filter users for a specific supervisor.

You can get the users associated to a specific supervisor (or co-supervisor) by checking related keys in `user.mila_ldap` dictionary. User email is available in `user.mila.email` attribute.

In [ ]:
# Choose a specific supervisor (here, sort supervisors and choose the last one).
specific_supervisor = sorted(supervisors)[-1]
print(f"Last sorted supervisor: {specific_supervisor}")
# Then, get supervised students.
students = [
    user.mila.email 
    for user in users 
    if (
        user.mila_ldap.get("supervisor") == specific_supervisor 
        or user.mila_ldap.get("co_supervisor") == specific_supervisor
    )
]
print(f"Number of students for last sorted supervisor: {len(students)}")